In [1]:
import os
import argparse
import requests
from dephell_licenses import licenses

In [2]:
class python_package():
    def __init__(self,
                 package: str,
                 directory: str = 'package',
                 package_registry: str = 'https://pypi.org/pypi',
                 metadata_format: str = 'json') -> None:
        self._package: str = package
        self._name: str = 'python-{}'.format(package.lower())
        self._package_dir: str = os.path.join(directory, self._name)
        if os.path.isdir(self._package_dir):
            print('Skipping {} as it already exists.'.format(self._package))
            raise IsADirectoryError('Directory {} already exists.'.format(self._package_dir))
        print('Configuring metadata of {} …'.format(self._package))
        os.mkdir(self._package_dir)
        metadata: str = requests.get(package_registry + '/' + package + '/' + metadata_format).json()
        self._info: dict = metadata['info']
        self._releases: dict = metadata['releases']
        self._urls: dict = metadata['urls']
        self._version: str = self._info['version']
        
        for url in self._urls:
            if url['packagetype'] == 'sdist':
                self._download_object: dict = url
        
        self._summary: str = self._info['summary']
        
        # every® project have this file in its root nowadays
        self._licensefile: str = 'LICENSE'
        self._license: None = None
        
        py2_compatible: bool = False
        py3_compatible: bool = False
        for classifier in self._info['classifiers']:
            if classifier == 'Programming Language :: Python :: 2':
                py2_compatible: bool = True
            if classifier == 'Programming Language :: Python :: 3':
                py3_compatible: bool = True
            if classifier.startswith('License'):
                self._license: licenses = licenses.get_by_classifier(classifier)
        if not self._license:
            self._license: licenses = licenses.get_by_id(self._info['license'])
        if not self._license:
            self._license: str = self._info['license']
        
        self._python_version: str = ''
        if py2_compatible and not py3_compatible:
            self._python_version: str = '2'
        if not py2_compatible and py3_compatible:
            self._python_version: str = '3'
        
        # even pip just calls stuptools independent of the used tool
        # https://docs.python.org/3/library/distutils.html
        self._setup_type: str = 'setuptools' 
        
        try:
            self._project_url: str = self._info['project_urls']['Homepage']
        except TypeError:
            self._project_url: str = self._info['project_url']
        
        # First split on ';', then the zeroth element on ' '
        try:
            self._dependencies: list = [ str.split(str.split(requirement, sep = ';')[0])[0] for requirement in self._info['requires_dist'] ]
        except TypeError:
            self._dependencies: list = []
        
        for url in self._urls:
            if url['packagetype'] == 'sdist':
                self._download_object: dict = url
        try:
            self._download_filename: str = self._download_object['filename']
            # remove the filename from the download path
            self._download_url: str = self._download_object['url'].rsplit(self._download_filename)[0]
            self._download_hash: dict = dict(self._download_object['digests'])
        except AttributeError:
            self._download_filename: str = 'Unknown'
            self._download_url: str = 'Unknown'
            self._download_hash: dict = {'Unknown': 'Unknown'}
        self.write_config_in()
        self.write_mk_file()
        self.write_hash_file()
        
        for dependency in self._dependencies:
            try:
                python_package(dependency)
            except IsADirectoryError:
                pass
    
    def _escape_for_makefile(self, package: str, prefix: str='', suffix: str='') -> str:
        return('{}{}{}'.format(prefix, package.upper().replace('-', '_').replace('.', ''), suffix))

    def write_config_in(self):
        dependencies: list = self._dependencies
        name: str = self._name
        package_dir: str = self._package_dir
        project_url: str = self._project_url
        python_version: str = self._python_version
        summary: str = self._summary
        with open(os.path.join(package_dir, 'Config.in'), 'w') as f:
            f.write('# vim: noexpandtab filetype=make\n')
            f.write('config {}\n'.format(self._escape_for_makefile(package=name, prefix='BR2_PACKAGE_')))
            f.write('\tbool "{}"\n'.format(name))
            if python_version:
                f.write('\tdepends on BR2_PACKAGE_PYTHON{}\n'.format(python_version.replace('2', '')))
            for dependency in dependencies:
                f.write('\tselect {}\n'.format(self._escape_for_makefile(package=dependency, prefix='BR2_PACKAGE_PYTHON_')))
            f.write('\thelp\n')
            f.write('\t  {}\n\n\t  {}\n'.format(summary, project_url))
            
    def write_hash_file(self):
        download_filename: str = self._download_filename
        hashes: dict = self._download_hash
        name: str = self._name
        package_dir: str = self._package_dir
        with open(os.path.join(package_dir, name + '.hash'), 'w') as f:
            for hash_type in hashes:
                f.write('{} {} {}\n'.format(hash_type, hashes[hash_type], download_filename))
    
    def write_mk_file(self):
        escaped_package: str = self._escape_for_makefile(self._package)
        try:
            license: str = self._license.id
        except AttributeError:
            license: str = self._license
        with open(os.path.join(self._package_dir, self._name + '.mk'), 'w') as f:
            f.write('# vim: noexpandtab filetype=make\n')
            f.write('#' * 80 + '\n')
            f.write('#\n')
            f.write('# {}\n'.format(self._name))
            f.write('#\n')
            f.write('#' * 80 + '\n')
            f.write('\n')
            f.write('PYTHON_{}_VERSION = {}\n'.format(escaped_package, self._version))
            f.write('PYTHON_{}_SOURCE = {}\n'.format(escaped_package, self._download_filename))
            f.write('PYTHON_{}_SITE = {}\n'.format(escaped_package, self._download_url))
            f.write('PYTHON_{}_LICENSE = {}\n'.format(escaped_package, license))
            f.write('PYTHON_{}_LICENSE_FILES = {}\n'.format(escaped_package, self._licensefile))
            f.write('PYTHON_{}_SETUP_TYPE = {}\n'.format(escaped_package, self._setup_type))
            f.write('\n')
            f.write('$(eval $(python-package))\n')
            

In [3]:
# Building the parser
parser: argparse.ArgumentParser = argparse.ArgumentParser(
    description="Creates buildroot packages from the metadata of "
                "an existing PyPI packages and include it "
                "in menuconfig")
parser.add_argument("packages",
                    help="list of packages to be created",
                    nargs='+')
parser.add_argument("-o", "--output",
                    help="""
                    Output directory for packages.
                    Default is ./package
                    """,
                    default='./package')

args: argparse.Namespace = parser.parse_args()
packages: list = list(set(args.packages))
    
for package in packages:
    python_package(package=package, directory=args.output)

usage: ipykernel_launcher.py [-h] [-o OUTPUT] packages [packages ...]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
